<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/Assignmnet_5_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##**Part 1 -- By Zilal**
- Design SVM From Scratch


In [7]:
DATA_DIR='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/A5/Data'

In [40]:
import numpy as np

class LinearSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.epochs = epochs

    def fit(self, X, y):
        # Initialize parameters
        self.w = np.random.randn(X.shape[1])  # Random initialization of weights
        self.b = 0  # Bias initialized to 0

        # Stochastic Gradient Descent
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                xi = X[i, :]
                yi = y[i]

                # Compute the hinge loss gradient
                if yi * (np.dot(xi, self.w) + self.b) < 1:
                    dw = self.w - yi * xi
                    db = -yi
                else:
                    dw = self.w
                    db = 0

                # Add regularization term (derivative of lambda * ||w||^2)
                dw += 2 * self.lambda_param * self.w

                # Update parameters
                self.w -= self.learning_rate * dw
                self.b -= self.learning_rate * db

            # Optionally, print the loss after each epoch to track progress
            loss = self.compute_loss(X, y)
            print(f"Epoch {epoch+1}/{self.epochs}, Loss: {loss}")

    def compute_loss(self, X, y):
        # Compute the hinge loss + regularization
        margins = 1 - y * (np.dot(X, self.w) + self.b)
        hinge_loss = np.sum(np.maximum(0, margins)) / X.shape[0]
        reg_loss = self.lambda_param * np.sum(self.w ** 2)
        return hinge_loss + reg_loss

    def predict(self, X):
        # Predict the class labels: 1 if positive, -1 if negative
        return np.sign(np.dot(X, self.w) + self.b)


Testing SVM

In [39]:
##Testing
# Example usage:
train = pd.read_csv(f'{DATA_DIR}/coding5_train.csv')
test = pd.read_csv(f'{DATA_DIR}/coding5_test.csv')
Y=train['Y']
X_train=train.drop(['Y'] , axis=1)
X_train = X_train.astype(float)

Y_test=test['Y']
X_test=test.drop(['Y'] , axis=1)
print(type(X_train))
display(type(X_test))
print(f'type Y is {(Y.info())}')
model = LinearSVM(learning_rate=0.001, lambda_param=0.01, epochs=1000)
model.fit(X_train, Y)
predictions = model.predict(X_test)
predictions


<class 'pandas.core.frame.DataFrame'>


pandas.core.frame.DataFrame

<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: Y
Non-Null Count  Dtype
--------------  -----
200 non-null    int64
dtypes: int64(1)
memory usage: 1.7 KB
type Y is None


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U2'), dtype('<U32')) -> None